In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


## 1) La capitalisation boursière en tant que taille moyenne de l'entreprise x nombre d'entreprises

In [3]:
market_cap = df_numfirm.multiply(df_avgsize, axis=0)

## 2) Le rapport entre la valeur comptable (BE : book equity) et la valeur de marché (ME : market equity), c'est-à-dire le rapport book-to-market, en utilisant les données "Sum of BE / Sum of ME".

### (Remarque : les données "Somme de BE / Somme de ME" sont annuelles et doivent être converties en données mensuelles en supposant que le ratio reste constant entre juillet de l'année s et juin de l'année s + 1. Par exemple, le ratio de 1926 est le ratio qui doit être utilisé pour la période allant de juillet 1926 à juin 1927).

In [4]:
df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

## 3) Le momentum de chaque industrie est le rendement moyen de cette industrie au cours des 12 derniers mois, y compris le mois t.

In [5]:
# Calculer la moyenne mobile sur 12 mois pour chaque industrie
df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

## Pour chaque mois t, standardiser chaque caractéristique de manière transversale pour avoir une moyenne nulle et un écart-type unitaire pour toutes les actions à la date t, comme expliqué dans Brandt et al. (2009).

In [6]:
# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_cap)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

In [7]:

standardized_mom.loc['1969-07-01':'1973-12-01']


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07-01,-0.689567,-0.988299,-0.637683,-0.604571,-0.794286,-0.515915,-0.957432,-1.427232,-0.469193,-1.271476,...,-0.268799,-1.373947,-1.443476,-0.454795,-0.028210,-0.959620,-0.679834,0.458288,-0.827547,-0.498115
1969-08-01,-0.432832,-0.781008,-0.599273,-0.667004,-0.456205,-0.368395,-0.845416,-1.030329,-0.304049,-1.365325,...,-0.192402,-1.110734,-1.133417,-0.145353,0.112186,-0.492252,-0.904375,0.659333,-0.625786,-0.134649
1969-09-01,-0.833021,-1.174734,-0.301005,-1.002437,-0.531072,-0.694606,-1.354300,-1.259834,-0.458518,-1.620113,...,-0.231146,-1.759102,-1.550324,-0.585914,-0.327849,-0.987718,-1.868762,0.085440,-0.987149,-1.044523
1969-10-01,-0.285915,-0.734873,-0.017911,-0.056299,-0.019713,-0.451398,-0.855380,-1.051717,-0.106882,-1.410053,...,-0.239877,-1.519681,-1.175857,-0.277521,-0.215249,-0.613074,-1.512808,0.167852,-0.731184,-0.618599
1969-11-01,-0.689072,-1.136815,-0.487837,-0.512625,-0.134133,-0.953400,-1.245032,-1.247906,-0.697128,-1.712365,...,-0.728819,-2.309322,-2.072581,-0.729097,-0.746683,-1.166142,-1.803055,-0.482471,-1.413042,-0.974409
1969-12-01,-0.618829,-1.176630,-0.179137,-0.517165,-0.710943,-1.041476,-1.209984,-1.184566,-0.200442,-1.918432,...,-0.638779,-2.503138,-2.231355,-0.615809,-1.018524,-0.790093,-1.880144,-0.665904,-1.267206,-2.026526
1970-01-01,-0.847862,-1.288491,-0.302902,-0.568246,-1.211472,-1.212918,-1.379383,-1.426409,-0.638731,-2.081369,...,-0.960739,-3.027583,-2.531428,-0.876475,-1.524989,-1.162864,-2.157974,-1.051048,-1.738268,-2.743383
1970-02-01,0.106360,-0.547173,0.116285,0.106594,-0.750496,-0.763472,-0.835451,-1.146726,-0.330422,-1.583772,...,-0.349016,-2.103113,-1.754533,-0.422801,-0.717944,-0.106369,-1.118568,-0.568206,-0.953164,-2.278373
1970-03-01,-0.858744,-0.578141,-0.120337,-0.023947,-0.602644,-1.008131,-0.776007,-1.421063,-0.528845,-1.637286,...,-0.841232,-2.122445,-2.172439,-0.763712,-1.236657,-0.357380,-0.753819,-0.905165,-1.174714,-2.349293


In [8]:
# Divisez chaque market cap par la somme totale correspondante
market_weights = market_cap.div(market_cap.sum(axis=1), axis=0)

optimal_weights = market_weights + (1 / len(market_weights.columns)) * (theta1 * standardized_market_caps + theta2 * standardized_BtoM + theta3 * standardized_mom)

NameError: name 'theta1' is not defined

In [ ]:

# Calculer la fonction à maximiser
# Vous pouvez remplacer les opérations par votre propre formule
function_to_maximize = (1/T) * np.sum(u * np.sum(wi_t + (1/N) * (ß_MC + ß_BM + ß_MOM)) * (1 + r))

# Afficher le résultat
print("Fonction à maximiser :", function_to_maximize)


In [ ]:
market_weights

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,0.011213,0.046726,-0.000000,0.000800,0.035787,0.000487,0.006436,0.000162,0.006673,0.008252,...,0.007944,0.186436,0.000089,0.057657,0.002431,0.001629,0.003575,0.001717,0.001858,0.003633
1926-08-01,0.011143,0.045346,-0.000000,0.000737,0.035211,0.000514,0.006407,0.000237,0.006443,0.008660,...,0.008267,0.184083,0.000066,0.055930,0.002402,0.001656,0.003456,0.001715,0.001698,0.003714
1926-09-01,0.011139,0.045478,-0.000000,0.000916,0.036457,0.000587,0.006213,0.000331,0.006030,0.008237,...,0.007862,0.187946,0.000068,0.054148,0.002336,0.001812,0.003465,0.001742,0.001665,0.003873
1926-10-01,0.010987,0.045853,-0.000000,0.000952,0.036682,0.000636,0.006530,0.000314,0.006044,0.008127,...,0.007425,0.187524,0.000063,0.054200,0.002306,0.001757,0.003487,0.001688,0.001582,0.003724
1926-11-01,0.011272,0.045377,-0.000000,0.000949,0.038228,0.000647,0.006390,0.000342,0.005915,0.008384,...,0.007225,0.187790,0.000054,0.054555,0.002275,0.001599,0.003443,0.001640,0.001614,0.003516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.000928,0.011433,0.007836,0.007835,0.005182,0.000942,0.008925,0.000620,0.015324,0.005710,...,0.001708,0.018376,0.011755,0.076022,0.015757,0.059433,0.035788,0.001558,0.027243,0.027591
2023-10-01,0.000986,0.011506,0.007677,0.007828,0.005236,0.000919,0.008347,0.000611,0.015038,0.005732,...,0.001748,0.018348,0.012045,0.075141,0.015854,0.060089,0.038577,0.001468,0.027638,0.028226
2023-11-01,0.000957,0.011142,0.007882,0.007718,0.005181,0.000809,0.008840,0.000616,0.015393,0.006156,...,0.001740,0.017716,0.012228,0.077994,0.016056,0.060001,0.041133,0.001374,0.026605,0.028574


# Code original

In [15]:
import numpy as np
from scipy.optimize import minimize

def u(x, gamma):
    return (x ** (1 - gamma) - 1) / (1 - gamma)

# Fonction à maximiser
def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
    wi_t = np.random.rand(N) # Supposons que vous avez une fonction pour calculer wi_t
    return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)), gamma) * (1 + r))

# Données
T = 54  # Nombre d'observations
N = 48   # Nombre d'actifs
MC = np.random.rand(T, N)  # Exemple de données de market cap
BM = np.random.rand(T, N)  # Exemple de données de book-to-market ratio
MOM = np.random.rand(T, N) # Exemple de données de momentum
r = np.random.rand(T)       # Exemple de données de rendement
gamma = 0.5  # Paramètre de la fonction d'utilité CRRA

# Initialisation des coefficients beta
initial_beta = np.array([0.5, 0.5, 0.5])

# Optimisation des coefficients beta
result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

# Résultats
optimal_beta = result.x
maximized_value = -result.fun

print("Coefficients β optimaux :", optimal_beta)
print("Valeur maximisée de la fonction :", maximized_value)


Coefficients β optimaux : [-145980.15708065  431762.43103814  284044.6232613 ]
Valeur maximisée de la fonction : 11824.726537663015


/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_3582/1715546139.py:5: RuntimeWarning: invalid value encountered in scalar power
  return (x ** (1 - gamma) - 1) / (1 - gamma)


# ******

In [18]:
def maximize_utility(MC, BM, MOM, r, T, N, gamma, start, end):
    
    def u(x, gamma):
        return (x ** (1 - gamma) - 1) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
        wi_t = market_weights.loc[start:end]
        return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC.loc[start:end] + beta[1] * BM.loc[start:end] + beta[2] * MOM.loc[start:end])), gamma) * (1 + r.loc[start:end]))

    # Initialisation des coefficients beta
    initial_beta = np.array([0.5, 0.5, 0.5])

    # Optimisation des coefficients beta
    result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

    # Résultats
    optimal_beta = result.x
    maximized_value = -result.fun

    return optimal_beta, maximized_value

start_date = pd.to_datetime('1969-07-01')
end_date = pd.to_datetime('1973-12-01')

maximize_utility(standardized_market_caps, standardized_BtoM, standardized_mom, df_48ind.shift(-1), 48, 48, 0.5, start_date, end_date)

ValueError: The user-provided objective function must return a scalar value.

In [19]:
def maximize_utility(MC, BM, MOM, r, T, N, gamma, start, end):
    
    def u(x, gamma):
        return (x ** (1 - gamma) - 1) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
        wi_t = market_weights.loc[start:end]
        return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)), gamma) * (1 + r))

    # Initialisation des coefficients beta
    initial_beta = np.array([0.5, 0.5, 0.5])

    # Optimisation des coefficients beta
    result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

    # Résultats
    optimal_beta = result.x
    maximized_value = -result.fun

    return optimal_beta, maximized_value

start_date = pd.to_datetime('1969-07-01')
end_date = pd.to_datetime('1973-12-01')

maximize_utility(standardized_market_caps, standardized_BtoM, standardized_mom, df_48ind, 48, 48, 5, start_date, end_date)

TypeError: maximize_utility.<locals>.u() missing 1 required positional argument: 'gamma'